In [6]:
import torch
from build_dataloader import get_train_loader, get_test_loader
from custom_resnet import CustomRes
from torchsummary import summary
from trainer import train, test
from torch.optim import SGD,Adam
from torch.optim.lr_scheduler import OneCycleLR

In [2]:
BATCHSIZE=512
train_dataloader = get_train_loader(data_dir='./data', 
                                    train=True,
                                    download=True,
                                    shuffle=True,
                                    batch_size=BATCHSIZE
                                    )

test_dataloader = get_test_loader(data_dir='./data', 
                                    train=False,
                                    download=True,
                                    shuffle=True,
                                    batch_size=BATCHSIZE
                                    )

Files already downloaded and verified
Files already downloaded and verified


In [3]:
model = CustomRes()
summary(model, input_size=(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
            Conv2d-4          [-1, 128, 32, 32]          73,728
         MaxPool2d-5          [-1, 128, 16, 16]               0
       BatchNorm2d-6          [-1, 128, 16, 16]             256
              ReLU-7          [-1, 128, 16, 16]               0
            Conv2d-8          [-1, 128, 16, 16]         147,456
       BatchNorm2d-9          [-1, 128, 16, 16]             256
             ReLU-10          [-1, 128, 16, 16]               0
           Conv2d-11          [-1, 128, 16, 16]         147,456
      BatchNorm2d-12          [-1, 128, 16, 16]             256
             ReLU-13          [-1, 128, 16, 16]               0
           Conv2d-14          [-1, 256,

In [4]:
EPOCHS = 25
optimizer = Adam(params=model.parameters(), lr=0.08)
scheduler = OneCycleLR(optimizer=optimizer, max_lr=.01, epochs=EPOCHS, steps_per_epoch=len(train_dataloader))
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device("mps")
for e in range(EPOCHS):
    train(model, optimizer=optimizer, train_dataloader=train_dataloader,device=device,epoch=e,scheduler=scheduler )
    test(model,test_dataloader=test_dataloader, device=device, epoch=e)

  0%|          | 0/98 [00:00<?, ?it/s]

100%|██████████| 98/98 [00:36<00:00,  2.66it/s]


at epoch:0
avg_train_loss:1.3833872079849243,avg_train_acc:0.5119104385375977
avg_test_loss:1.027896523475647,avg_test_acc:0.634409487247467


100%|██████████| 98/98 [00:36<00:00,  2.67it/s]


at epoch:1
avg_train_loss:0.8473235964775085,avg_train_acc:0.7014414072036743
avg_test_loss:0.7971357107162476,avg_test_acc:0.7290154099464417


...

at epoch:22
avg_train_loss:0.05962052196264267,avg_train_acc:0.9801954030990601
avg_test_loss:0.49754634499549866,avg_test_acc:0.8882295489311218


100%|██████████| 98/98 [00:35<00:00,  2.78it/s]


at epoch:23
avg_train_loss:0.059625908732414246,avg_train_acc:0.9810552597045898
avg_test_loss:0.49563026428222656,avg_test_acc:0.8920725584030151


100%|██████████| 98/98 [00:36<00:00,  2.67it/s]


at epoch:24
avg_train_loss:0.05672013387084007,avg_train_acc:0.9825699329376221
avg_test_loss:0.49775201082229614,avg_test_acc:0.8910788297653198
